In [50]:
import pandas as pd

In [52]:
# Install a pip package in the current Jupyter kernel :)
import sys
!{sys.executable} -m pip install openpyxl

     |████████████████████████████████| 174kB 1.3MB/s eta 0:00:01
  Created wheel for openpyxl: filename=openpyxl-3.0.2-py2.py3-none-any.whl size=241181 sha256=d200a6fb2d859168575464845d66a793000857f48bf97c8f293d0c6d02db6718
  Stored in directory: /root/.cache/pip/wheels/41/ba/1e/266431033670ab5b010a3eddba51cf1a6dc5cbc7231e32f464
  Created wheel for et-xmlfile: filename=et_xmlfile-1.0.1-cp37-none-any.whl size=8917 sha256=5ed9e04a6ebf4176eeb83d3812db77af1e75d99b650279f03a65567e96f6f42d
  Stored in directory: /root/.cache/pip/wheels/2a/77/35/0da0965a057698121fc7d8c5a7a9955cdbfb3cc4e2423cad39
Successfully built openpyxl et-xmlfile


In [3]:
from IPython.display import display
from IPython.display import HTML
import IPython.core.display as di # Example: di.display_html('<h3>%s:</h3>' % str, raw=True)

# This line will hide code by default when the notebook is exported as HTML
# di.display_html('<script>jQuery(function() {if (jQuery("body.notebook_app").length == 0) { jQuery(".input_area").toggle(); jQuery(".prompt").toggle();}});</script>', raw=True)

display(HTML("<style>.container { width:100% !important; }</style>"))

In [378]:
csv_list = !ls 'sum'*'csv'

In [379]:
csv_list

['sum_adhd.csv',
 'sum_agg_beh.csv',
 'sum_bg.csv',
 'sum_echolalia.csv',
 'sum_gts_genes.csv',
 'sum_inv_mov.csv',
 'sum_neuro_dev.csv',
 'sum_ocd_beh.csv',
 'sum_patho.csv',
 'sum_self_mut.csv',
 'sum_tics.csv']

In [390]:
dfs = []
dataframes = []

for f in csv_list:
    dfs.append(pd.read_csv(f))
    dataframes.append(pd.read_csv(f))

### This is a script to open .csv generated by imdik-zekanowski-gts, to remove some characters, split SnipEff annotations and make an excel file with separate sheets

In [391]:
for idx, df in enumerate(dfs):
    
    df.replace({'\[|\]|\'':''}, regex = True, inplace = True) 
    df.replace({'\|':' '}, regex = True, inplace = True)  
    df.replace({'\^':' '}, regex = True,inplace = True)
   
    dataframes[idx] = df.SnpEff.str.split(" ", expand=True)
    dataframes[idx] = dataframes[idx].iloc[:, [1,2,7,10]]
    dataframes[idx].columns = ['variant_type','impact','location','AA']
    
    df.drop(df.columns[len(df.columns)-1], axis=1, inplace=True)
    df.drop(df.columns[0], axis = 1, inplace = True)

    dfs[idx] = pd.concat([df, dataframes[idx]], axis=1)
    
    d = dfs[0:(len(dfs)-1)]
    d = pd.concat(d)
    d = d[['sample','locus', 'genotype', 'alleles', 'family', 'sex', 'additional_pheno', 'Gene', 'impact']]
    d.sort_values(['family', 'sample'], axis=0, inplace=True, ascending=True)
    
dfs.append(pd.read_csv('GTS-coded.csv'))


with pd.ExcelWriter('rare_patients_only.xlsx') as writer:
    for idx, df in enumerate(dfs):
        try:
            df.to_excel(writer, sheet_name=str(csv_list[idx]))
        except IndexError:
            df.to_excel(writer, sheet_name='GTS-coded')
    d.to_excel(writer, sheet_name='sample_centric')

/usr/local/lib/python3.7/site-packages/ipykernel_launcher.py:17: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

